In [7]:
import numpy as np

# Choose the filename of the uploaded file
filename_true = "/content/drive/MyDrive/word_data/output.txt"

# Read the content of the file line by line and store it in a NumPy array
lines = []
with open(filename_true, 'r') as file:
    for line in file:
        lines.append(line.strip())

# Convert the list of lines to a NumPy array
np_true = np.array(lines)

filename_true_labels= "/content/drive/MyDrive/word_data/output_label.txt"
lines = []
with open(filename_true_labels, 'r') as file:
    for line in file:
        lines.append(line.strip())

np_true_label= np.array(lines)


filename_false= "/content/drive/MyDrive/word_data/false_data.txt"
lines = []
with open(filename_false, 'r') as file:
    for line in file:
        lines.append(line.strip())
np_false= np.array(lines)


filename_false_labels= "/content/drive/MyDrive/word_data/false_data_labels.txt"
lines = []
with open(filename_false_labels, 'r') as file:
    for line in file:
        lines.append(line.strip())

np_false_label= np.array(lines)



In [8]:
filename_true2 = "/content/drive/MyDrive/word_data2/output2.txt"

# Read the content of the file line by line and store it in a NumPy array
lines = []
with open(filename_true2, 'r') as file:
    for line in file:
        lines.append(line.strip())

# Convert the list of lines to a NumPy array
np_true2 = np.array(lines)

filename_true_labels2= "/content/drive/MyDrive/word_data2/output_label2.txt"
lines = []
with open(filename_true_labels2, 'r') as file:
    for line in file:
        lines.append(line.strip())

np_true_label2= np.array(lines)


filename_false2= "/content/drive/MyDrive/word_data2/false_data2.txt"
lines = []
with open(filename_false2, 'r') as file:
    for line in file:
        lines.append(line.strip())
np_false2= np.array(lines)


filename_false_labels2= "/content/drive/MyDrive/word_data2/false_data_labels2.txt"
lines = []
with open(filename_false_labels2, 'r') as file:
    for line in file:
        lines.append(line.strip())

np_false_label2= np.array(lines)

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming X_total is your input data and y_total is your labels

# Concatenate your data
X_total = np.concatenate((np_true, np_false), axis=0)
y_total = np.concatenate((np_true_label, np_false_label), axis=0)
X_total2 = np.concatenate((np_true2, np_false2), axis=0)
y_total2 = np.concatenate((np_true_label2, np_false_label2), axis=0)
# Calculate the size of the reduced dataset (1/10)
fraction_to_keep = 0.9  # 10% of the data
fraction_to_keep2 = 0.9
# Use train_test_split to randomly select a subset
X_reduced, _, y_reduced, _ = train_test_split(X_total, y_total, test_size=1-fraction_to_keep, random_state=42)
X_reduced2, _, y_reduced2, _ = train_test_split(X_total2, y_total2, test_size=1-fraction_to_keep2, random_state=42)

y_reduced=y_reduced.astype(int)
y_reduced2=y_reduced2.astype(int)
# Split the reduced data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y_reduced, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_reduced2, y_reduced2, test_size=0.2, random_state=42)

In [10]:
X_train = np.concatenate((X_train,X_train2), axis=0)
y_train = np.concatenate((y_train,y_train2), axis=0)


In [ ]:
test_x=X_test

In [11]:
from gensim.models import Word2Vec

sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, window=5, min_count=5, workers=4)

In [12]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(sentence) for sentence in X_train])
X_test = np.array([vectorize(sentence) for sentence in X_test])
X_test2 = np.array([vectorize(sentence) for sentence in X_test2])
#max_len = max(len(sentence.split()) for sentence in X_train)
#X_train = pad_sequences(X_train, maxlen=100, padding='post')
#X_test = pad_sequences(X_test, maxlen=100, padding='post')


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Assuming `X_train` is your input data and `y_train` is your labels
#max_len = max(len(sentence.split()) for sentence in X_train)
# Text Vectorization
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Model Architecture
# Create a Sequential model
model = Sequential()

# Add a Dense layer for classification
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))
#model.add(LSTM(units=64, activation='relu'))
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test2,y_test2))

# Evaluate on the test set
y_test_pred = model.predict(X_test)
y_test_pred_binary = (y_test_pred > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred_binary)
print(f"Test Accuracy: {accuracy}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_test_pred_binary)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_test_pred_binary)
recall = recall_score(y_test, y_test_pred_binary)
f1 = f1_score(y_test, y_test_pred_binary)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
# Evaluate the model
# Assuming `X_test` is your test data and `y_test` is your test labels


Epoch 1/5
36910/36910 [==============================] - 176s 5ms/step - loss: 0.2402 - accuracy: 0.8937 - val_loss: 0.2556 - val_accuracy: 0.8853
Epoch 2/5
36910/36910 [==============================] - 163s 4ms/step - loss: 0.2184 - accuracy: 0.9040 - val_loss: 0.2459 - val_accuracy: 0.8902
Epoch 3/5
36910/36910 [==============================] - 180s 5ms/step - loss: 0.2126 - accuracy: 0.9070 - val_loss: 0.2481 - val_accuracy: 0.8898
Epoch 4/5
36910/36910 [==============================] - 178s 5ms/step - loss: 0.2093 - accuracy: 0.9085 - val_loss: 0.2433 - val_accuracy: 0.8919
Epoch 5/5
3443/3443 [==============================] - 12s 3ms/step
Test Accuracy: 0.9338870733478577
Confusion Matrix:
[[52155  3117]
 [ 4166 50722]]
Precision: 0.9421051654005461
Recall: 0.9240999854248652
F1 Score: 0.9330157182668518


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
for i in range(len(X_test)):
  print(f"Test:{test_x[i]} Prediction:{y_test_pred_binary[i]}")

Test:Bunu yapmak için site yöneticisi, genellikle her kullanıcı için bir sertifika oluşturur ve tarayıcıya yüklenen bir sertifika oluşturur.Normalde bu, yetkili kullanıcının adını ve e-posta adresini içerir ve kullanıcının kimliğini doğrulamak için, muhtemelen bir parola bile girmeden, her yeniden bağlantıda sunucu tarafından otomatik olarak kontrol edilir. Prediction:[1]
Test:Bu tarihi izleyen yıllarda yöredeki kentleri yerle bir eden şiddetli bir depremden sonra Iğdır şehri bugünkü yerinde 1664'te kuruldu. Prediction:[1]
Test:Mutlaki ve meşruti bir hükumetde Prediction:[1]
Test:Osmanlı Devleti’nin kuruluş aşamasında da Bâtınîler, öncelikle askeri kuvvetlerin içerisinde yer almayı ihmâl etmediler. Prediction:[1]
Test:Bu miktarın büyük çoğunluğu Dublin Liman Tüneline, yedi Luas projesine, iki Dublin metro hattına, DART uzantılarına ve bütün servisleri St Stephen's Green’de bir araya getirecek bir yeraltı istasyonuna harcanacaktır. Prediction:[1]


IndexError: index 5 is out of bounds for axis 0 with size 5

In [ ]:
strings_list = ["Sizde mi geleceksiniz", "Bütün eşyalar arabada duruyor", "kitaplarınızı çantanızda taşıyın",
                "sınava hesap makineside getirebilirsiniz", "Bilgisayarımda antivirüs programı yüklü"
                ,"Kenan abide aşı olmuş","kedilerde havada uçamazmış","adamı uyurkende rahatsız etme","kaleminde senin gibi güzel",
                "banada şu eriklerden versene","Bu işde sana göre değilmiş",
                "Datasetleri güzel seçelimde yapay zeka doğru çalışsın","Bilgisayarın ekranınıda kırmış",
                "Yemekden sonra kahvemide içtim","ellerini yıkadıktan sonra dişlerinide fırçala",
                "Gelecek etkinliklerimizede sizi bekleriz","benimlede dışarı çıkar mısın"
                ,"Televizyon izlemektende sıkılmış","Bu borsa işlerine bizide bulaştırdı","arkadaşına yaptıklarına bende inanamadım"]

# Create a NumPy array from the list of strings
test_labels=[0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
test_labels=np.array(test_labels)
my_sentences = np.array(strings_list)

In [ ]:
test_my_sentences= np.array([vectorize(sentence) for sentence in my_sentences])
y_test_pred = model.predict(test_my_sentences)
y_test_pred_binary = (y_test_pred > 0.5).astype(int)
accuracy=accuracy_score(test_labels,y_test_pred_binary)
print(f"Test Accuracy: {accuracy}")
# Calculate accuracy
for i in range(len(my_sentences)):
  print(f"Test:{my_sentences[i]} Prediction:{y_test_pred_binary[i]}")

1/1 [==============================] - 2s 2s/step
Test Accuracy: 0.85
Test:Sizde mi geleceksiniz Prediction:[0]
Test:Bütün eşyalar arabada duruyor Prediction:[1]
Test:kitaplarınızı çantanızda taşıyın Prediction:[0]
Test:sınava hesap makineside getirebilirsiniz Prediction:[0]
Test:Bilgisayarımda antivirüs programı yüklü Prediction:[1]
Test:Kenan abide aşı olmuş Prediction:[1]
Test:kedilerde havada uçamazmış Prediction:[0]
Test:adamı uyurkende rahatsız etme Prediction:[0]
Test:kaleminde senin gibi güzel Prediction:[0]
Test:banada şu eriklerden versene Prediction:[0]
Test:Bu işde sana göre değilmiş Prediction:[0]
Test:Datasetleri güzel seçelimde yapay zeka doğru çalışsın Prediction:[0]
Test:Bilgisayarın ekranınıda kırmış Prediction:[0]
Test:Yemekden sonra kahvemide içtim Prediction:[0]
Test:ellerini yıkadıktan sonra dişlerinide fırçala Prediction:[0]
Test:Gelecek etkinliklerimizede sizi bekleriz Prediction:[0]
Test:benimlede dışarı çıkar mısın Prediction:[1]
Test:Televizyon izlemektende s